In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# import the trainingset
data = pd.read_csv('letter-recognition.csv',header=None)  
data.head(5)

In [17]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KernelDensity as KDe
from sklearn.ensemble import GradientBoostingRegressor

In [31]:
# prepare the trainging data and test data
data = pd.read_csv('letter-recognition.csv',header=None) 
#X = data[data.columns[1:]].values
#y = data[0].values
#print(X)
#print(y)
#cols = data.columns.tolist()
#data.drop_duplicates(subset=cols[:-1], inplace=True)

# pair H and K
data_h = data[data[0] == 'H']
data_k = data[data[0] == 'K']
data1 = pd.concat([data_h, data_k], axis=0)
data1.loc[data1[0] == 'H', 0] = 0
data1.loc[data1[0] == 'K', 0] = 1 

#imputer = KNNImputer(n_neighbors=1)
#data1 = imputer.fit_transform(data1)

X = data1[data1.columns[1:]]
X.reset_index(drop=True, inplace=True)
y = data1[0]
y.reset_index(drop=True, inplace=True)
print(X)
print(y)

cv = KFold(n_splits=10, random_state=1, shuffle=True)

      1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16
0      4   5   5   4   4   7   7   6   6   7   6   8   3   8   3   8
1      6   9   8   7   6   8   6   6   7   7   7   9   6   8   4   8
2      3   3   4   1   2   8   7   5   6   7   6   8   5   8   3   7
3      3   5   5   4   3   7   8   3   6  10   6   8   3   8   3   8
4      8  12   8   6   4   9   8   4   5   8   4   5   6   9   5   9
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
1468   7   9   8   5   4   5   8   4   6  10  11  11   6  12   4   7
1469   5   9   5   4   2   7   7   3   6   9   8   9   6  11   3   7
1470   7  14   8   8   5   9   6   3   5  11   3   7   5   7   4   8
1471   3   1   5   3   3   6   7   4   8   7   6  11   3   8   5   9
1472   5  11   5   8   5   3   8   7   3   6   4  11   3   8   2  11

[1473 rows x 16 columns]
0       0
1       0
2       0
3       0
4       0
       ..
1468    1
1469    1
1470    1
1471    1
1472    1
Name: 0, Length: 1473, dtype: object

In [26]:
# cross validation
def cross_validation(model):
    in_sample_error = []
    out_sample_error = []
    err_zscore = []

    for train_index, val_index in cv.split(X):
        Xtrain, Xval = X[train_index], X[val_index]
        ytrain, yval = y[train_index], y[val_index]
        model.fit(Xtrain,ytrain)

        pred_train,pred_val = model.predict(Xtrain),model.predict(Xval)
        pred = np.concatenate((pred_train, pred_val), axis=None)
        true_label = np.concatenate((ytrain,yval),axis=None)
        error = np.square(pred - true_label)/true_label.shape[0]
        #error = pred - true_label
        err_zscore.append(stats.zscore(error).reshape(-1,1))
        in_sample_error.append(mean_squared_error(pred_train, ytrain, squared=True))
        out_sample_error.append(mean_squared_error(pred_val, yval, squared=True))

    in_sample_error = np.array(in_sample_error)
    out_sample_error = np.array(out_sample_error)
    return in_sample_error,out_sample_error, err_zscore


* Random Forest

In [38]:
RF = RandomForestRegressor(max_depth=4, random_state=0)
RF_in_sample_err,RF_out_sample_err, RF_zscore_err=cross_validation(RF)
# print(np.mean(RF_out_sample_err))
kde = KDe(bandwidth=1.0, kernel='gaussian')
kde.fit(RF_zscore_err)
lp = kde.score_samples(RF_zscore_err)
print(lp.shape)
sns.kdeplot(RF_zscore_err[:,0])

KeyError: '[0, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 105, 106, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 226, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 249, 